# 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/ML_Project

Mounted at /content/drive
/content/drive/MyDrive/ML_Project


# 모델 및 벡터 로드

In [2]:
import joblib
import pandas as pd

# Logistic Regression 최종 모델 로드
model = joblib.load("final_model.pkl")
tfidf = joblib.load("tfidf_vectorizer.pkl")

print("모델 및 벡터 로드 완료")

모델 및 벡터 로드 완료


# 전처리(기존과 동일)

In [3]:
import re

# Training 때 사용한 stopword 리스트 그대로 복붙
stopwords = set([
    # 1) filler / 말버릇
    "어", "음", "자", "뭐", "요", "네", "막", "그냥",
    "근데", "그죠", "거죠", "예",

    # 2) 지시어 (정보 없음)
    "이거", "이게", "이건", "그거", "그게",
    "저거", "요거", "요게", "얘는",

    # 3) 공손/형식적 표현
    "합니다", "되었습니다", "됩니다", "하겠습니다",
    "해주세요", "드릴", "보겠습니다", "할게요",
    "있습니다", "있어요", "있죠",

    # 4) 기능어 (기본 조사)
    "이", "그", "저", "을", "를", "은", "는", "에",
    "에서", "로", "것", "거", "건", "것들",
])

def preprocess(text):
    text = str(text).lower()                      # 소문자 변환
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", " ", text)  # 한글/영문/숫자 제외 모두 제거
    text = re.sub(r"\s+", " ", text).strip()      # 중복 공백 제거

    # 토큰화
    tokens = text.split()

    # Stopword 제거
    tokens = [t for t in tokens if t not in stopwords]

    # Training과 완전히 동일하게 문장 재구성
    return " ".join(tokens)

# 단일 문장 추론

In [4]:
def predict_sentence(sentence, threshold=0.06):
    clean = preprocess(sentence)
    vec = tfidf.transform([clean])
    prob_1 = model.predict_proba(vec)[0][1]
    pred = 1 if prob_1 >= threshold else 0
    return pred, prob_1

# 여러 문장 리스트 추론

In [5]:
def predict_batch(sentences, threshold=0.06):
    clean_list = [preprocess(s) for s in sentences]
    vec = tfidf.transform(clean_list)
    prob_1 = model.predict_proba(vec)[:, 1]
    preds = (prob_1 >= threshold).astype(int)
    return preds, prob_1

# 테스트

In [6]:
example_sentences = [
    "이번 알고리즘의 핵심 아이디어는 데이터를 반복적으로 갱신하는 것입니다.",
    "이 모델의 목적은 입력 문장에서 핵심 정보를 자동으로 추출하는 것입니다.",
    "여기까지 첫 번째 파트 설명이었습니다.",
    "혹시 질문 있으신가요?"
]

preds, probs = predict_batch(example_sentences, threshold=0.06)

for s, p, pr in zip(example_sentences, preds, probs):
    print(f"[문장] {s}")
    print(f"=> 예측 라벨: {p} (확률: {pr:.4f})")
    print("   • 1 = 핵심 문장 / 0 = 비핵심 문장\n")

[문장] 이번 알고리즘의 핵심 아이디어는 데이터를 반복적으로 갱신하는 것입니다.
=> 예측 라벨: 1 (확률: 0.1262)
   • 1 = 핵심 문장 / 0 = 비핵심 문장

[문장] 이 모델의 목적은 입력 문장에서 핵심 정보를 자동으로 추출하는 것입니다.
=> 예측 라벨: 1 (확률: 0.0617)
   • 1 = 핵심 문장 / 0 = 비핵심 문장

[문장] 여기까지 첫 번째 파트 설명이었습니다.
=> 예측 라벨: 0 (확률: 0.0329)
   • 1 = 핵심 문장 / 0 = 비핵심 문장

[문장] 혹시 질문 있으신가요?
=> 예측 라벨: 0 (확률: 0.0080)
   • 1 = 핵심 문장 / 0 = 비핵심 문장



In [7]:
while True:
    s = input("문장을 입력하세요 (종료: quit): ")
    if s.lower() == "quit":
        break
    pred, prob = predict_sentence(s)
    print(f"예측 라벨: {pred} (확률: {prob:.4f})  /  1=핵심, 0=비핵심\n")

문장을 입력하세요 (종료: quit): 분할 정복(Divide and Conquer) 알고리즘은 하나의 큰 문제를 작은 하위 문제로 분할(Divide)하고, 각 하위 문제를 해결(Conquer)한 뒤, 그 해결책들을 합쳐서(Combine) 원래의 문제를 해결하는 알고리즘 설계 기법입니다.
예측 라벨: 1 (확률: 0.2749)  /  1=핵심, 0=비핵심

문장을 입력하세요 (종료: quit): 여러분들 전부 이해하셨죠?
예측 라벨: 0 (확률: 0.0502)  /  1=핵심, 0=비핵심

문장을 입력하세요 (종료: quit): 아 네 그러면 여기까지 하겠습니다.
예측 라벨: 0 (확률: 0.0254)  /  1=핵심, 0=비핵심

문장을 입력하세요 (종료: quit): 잠시만요, 화면이 안 보이네요.
예측 라벨: 0 (확률: 0.0081)  /  1=핵심, 0=비핵심

문장을 입력하세요 (종료: quit): quit


# 핵심 문장 자동 추출기 (프로젝트 데모용)

In [12]:
import re

# 문장 단위로 자동 split
def split_sentences(text):
    # 마침표, 물음표, 느낌표 기준으로 문장 분리
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    # 빈 문장 제거
    return [s.strip() for s in sentences if s.strip()]

# 긴 텍스트 한 번에 입력 → 문장 자동 분리 → 핵심만 추출
def extract_key_from_text(threshold=0.06):
    print("="*70)
    print("강의자료 자동 요약 — 긴 텍스트 입력 모드")
    print("="*70)
    print("여러 문장을 한 번에 붙여넣으세요. (종료: 빈 줄 + Enter)")
    print("-" * 70)

    print("텍스트 입력 시작 ↓")
    text_lines = []
    while True:
        line = input()
        if line.strip() == "":
            break
        text_lines.append(line)

    full_text = "\n".join(text_lines).strip()

    if len(full_text) == 0:
        print("입력된 텍스트가 없습니다.")
        return

    # 문장 자동 분리
    sentences = split_sentences(full_text)
    print(f"\n총 {len(sentences)}개 문장으로 분리되었습니다.")

    preds, probs = predict_batch(sentences, threshold=threshold)

    # 핵심 문장만 추출
    key_sents = [(s, pr) for s, p, pr in zip(sentences, preds, probs) if p == 1]

    print("\n" + "="*70)
    print("추출된 핵심 문장")
    print("="*70)

    if not key_sents:
        print("핵심 문장이 없습니다. (threshold =", threshold, ")")
        return

    for idx, (s, pr) in enumerate(key_sents, 1):
        print(f"\n[{idx}] {s}")
        print(f"     ↳ 확률: {pr:.4f}")

    print("\n" + "="*70)
    print("✔ 총", len(key_sents), "개의 핵심 문장을 자동으로 추출했습니다.")
    print("="*70)

# 실행
extract_key_from_text(threshold=0.06)

강의자료 자동 요약 — 긴 텍스트 입력 모드
여러 문장을 한 번에 붙여넣으세요. (종료: 빈 줄 + Enter)
----------------------------------------------------------------------
텍스트 입력 시작 ↓
오늘은 경사하강법(Gradient Descent)의 핵심 아이디어를 살펴보겠습니다. 경사하강법은 비용 함수를 최소화하기 위해 기울기 방향으로 파라미터를 갱신하는 최적화 알고리즘입니다. 즉, 기울기가 큰 방향은 손실이 빠르게 감소하는 방향을 의미합니다. 이 과정에서 학습률(learning rate)은 이동하는 보폭을 조절하는 중요한 하이퍼파라미터입니다. 학습률이 너무 크면 발산할 수 있고, 너무 작으면 학습 속도가 매우 느려집니다. 여기서 혹시 질문 있으신가요? 자 그럼 간단한 예제를 보면서 이해해봅시다. 다음 그래프는 비용 함수의 곡선을 나타냅니다. 우리가 원하는 것은 이 곡선의 최솟값을 찾는 것입니다.


총 9개 문장으로 분리되었습니다.

추출된 핵심 문장

[1] 오늘은 경사하강법(Gradient Descent)의 핵심 아이디어를 살펴보겠습니다.
     ↳ 확률: 0.2866

[2] 이 과정에서 학습률(learning rate)은 이동하는 보폭을 조절하는 중요한 하이퍼파라미터입니다.
     ↳ 확률: 0.0977

[3] 학습률이 너무 크면 발산할 수 있고, 너무 작으면 학습 속도가 매우 느려집니다.
     ↳ 확률: 0.0620

[4] 우리가 원하는 것은 이 곡선의 최솟값을 찾는 것입니다.
     ↳ 확률: 0.2284

✔ 총 4 개의 핵심 문장을 자동으로 추출했습니다.


위 데모 셀에서 입력한 문장들:

오늘은 경사하강법(Gradient Descent)의 핵심 아이디어를 살펴보겠습니다.
경사하강법은 비용 함수를 최소화하기 위해 기울기 방향으로 파라미터를 갱신하는 최적화 알고리즘입니다.
즉, 기울기가 큰 방향은 손실이 빠르게 감소하는 방향을 의미합니다.
이 과정에서 학습률(learning rate)은 이동하는 보폭을 조절하는 중요한 하이퍼파라미터입니다.
학습률이 너무 크면 발산할 수 있고, 너무 작으면 학습 속도가 매우 느려집니다.
여기서 혹시 질문 있으신가요?
자 그럼 간단한 예제를 보면서 이해해봅시다.
다음 그래프는 비용 함수의 곡선을 나타냅니다.
우리가 원하는 것은 이 곡선의 최솟값을 찾는 것입니다.

내가 생각하는 핵심문장:

1. 경사하강법은 비용 함수를 최소화하기 위해 기울기 방향으로 파라미터를 갱신하는 최적화 알고리즘입니다.(불일치)

2. 즉, 기울기가 큰 방향은 손실이 빠르게 감소하는 방향을 의미합니다.(불일치)

3. 이 과정에서 학습률(learning rate)은 이동하는 보폭을 조절하는 중요한 하이퍼파라미터입니다.(일치)

4. 학습률이 너무 크면 발산할 수 있고, 너무 작으면 학습 속도가 매우 느려집니다.(일치)

-> 약 50% 일치